# MET Data Processing to FACEMDS Format
=============================
by Bharat Sharma and Anthony Walker

## Requisites
The processed files from the plot data are saved at `/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/` <br>
There are two files: <br>
1. `Processed_Duke_Met_Data_All_Vars_30m_FV.csv`: FillValue/Missing = -6999.0
2. `Processed_Duke_Met_Data_All_Vars_30m.csv`: FillValue/Missing = NaN or empty

In [1]:
# importing libraries
import xarray as xr
import glob
from datetime import datetime
import cftime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# paths

paths = {}
paths ["ELM-DUKE"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/ELM_Data/data/atm/datm7/CLM1PT_data/1x1pt_US-DUK/"
paths ["FACEMDS_Walker2018"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Walker_2018_FATES_MDS/data/"
paths ["DukeFACE_Oren2022"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_DUKE_Met/data/"
paths ["Save_Processed"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/"

In [3]:
# Read the Processed Data
df_all_vars_30m_FV = pd.read_csv(f"{paths['Save_Processed']}Processed_Duke_Met_Data_All_Vars_30m_FV.csv")
df_all_vars_30m = pd.read_csv(f"{paths['Save_Processed']}Processed_Duke_Met_Data_All_Vars_30m.csv")

## Creating DUKE_forcing_h.txt

In [86]:
# Dictionary of columns and their descriptions
dict_cols = {
'YEAR':'Year of measurement',
'DTIME':'Fractional day of year',
'DOY':'Day of year',
'HRMIN':'Hour:minute, marked at the middle of measurement interval with last two digits as minute',
'Rainf':'Total Precipitation over a time step of measurement, mm',
'Rainf_f': 'gap-filling flag, 0 = measured, 1 = derived from other variables, 2 = filled by \
mean diurnal cycle within 5-15 days, 3 = filled by data from nearby weather station, 4 = \
filled by using NARR (North American Regional Reanalysis) data',
'Tair':'Mean air temperature over a time step of measurement, deg Celsius',
'Tair_f':'gap-filling flag',
'RH':'Mean relative humidity over a time step of measurement',
'RH_f':'gap-filling flag',
'VPD':'Vapor pressure deficit, kPa',
'VPD_f':'gap-filling flag',
'PAR':'Incident or downward photosynthetically active radiation, umol/m2/s',   
'PAR_f':'gap-filling flag',    
'SM':'Soil Moisture integrates measurements from 0 to 30cm depth',
'SM_f':'gap-filling flag', 
'SWP':' Soil Water Potential',
'SWP_f':'gap-filling flag', 
'SVP':'Saturated Vapor Pressure',
'SVP_f':'gap-filling flag', 
'Rn':'Net Radiation',
'Rn_f':'gap-filling flag', 
'SLT':'Soil Temperature',   
'SLT_f':'gap-filling flag', 
}

In [87]:
dict_units = {
    'Rainf':'mm',
    'Tair':'deg C',
    'RH':'',
    'VPD':'kPa',
    'PAR':'umol/m2/s',
    'SM':'',
    'SWP':'',
    'SVP':'kPa',
    'Rn':'umol/m2/s',
    'SLT':'deg C'
}



In [21]:
df_h = pd.DataFrame(columns=dict_cols.keys())
df_h['YEAR'] = df_all_vars_30m['Year'].astype(int)
df_h['DOY'] = df_all_vars_30m['DOY'].astype(int)
df_h['HRMIN'] = df_all_vars_30m['Time'].astype(int)
df_h['DTIME'] = (df_all_vars_30m['Time']/(24*60)+df_all_vars_30m['DOY']).astype(float)
df_h[list(dict_units.keys())] = df_all_vars_30m[list(dict_units.keys())]
gap_fill_cols = [col + '_f' for col in list(dict_units.keys())]
df_h[gap_fill_cols] = 0

#DROPING THE LAST ROW
df_h.drop(df_h.index[-1], inplace=True)

# Adding the unit row below first row
unit_row = pd.Series([dict_units.get(col, '') for col in df_h.columns], index=df_h.columns)
df_h_save = pd.concat([pd.DataFrame([unit_row]), df_h.iloc[:]]).reset_index(drop=True)

df_h_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h.csv")
df_h_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h.txt")
fill_value = -6999.
df_h_fv_save = df_h_save.fillna(fill_value)
df_h_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h_fv.csv")
df_h_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h_fv.txt")

### With Time Index

In [22]:
df_h_wTime = df_h.copy(deep=True)
df_h_wTime['Time'] = df_all_vars_30m['Date']
df_h_wTime['Time'] = pd.to_datetime(df_h_wTime['Time'])
df_h_wTime = df_h_wTime.set_index('Time')
df_h_wTime

,YEAR,DTIME,DOY,HRMIN,Rainf,Rainf_f,Tair,Tair_f,RH,RH_f,...,SM,SM_f,SWP,SWP_f,SVP,SVP_f,Rn,Rn_f,SLT,SLT_f
Time,,,,,,,,,,,,,,,,,,,,,
1997-01-01 00:30:00,1997,1.020833,1,30,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 01:00:00,1997,1.069444,1,100,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 01:30:00,1997,1.090278,1,130,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 02:00:00,1997,1.138889,1,200,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 02:30:00,1997,1.159722,1,230,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31 21:30:00,2012,367.479167,366,2130,0.0,0,7.98,0,0.42,0,...,0.39,0,488.93,0,1.07,0,-43.54,0,10.4,0
2012-12-31 22:00:00,2012,367.527778,366,2200,0.0,0,7.58,0,0.44,0,...,0.39,0,488.87,0,1.04,0,-44.50,0,10.4,0
2012-12-31 22:30:00,2012,367.548611,366,2230,0.0,0,7.08,0,0.46,0,...,0.39,0,488.87,0,1.01,0,-41.98,0,10.4,0


## Creating DUKE_forcing_d.txt


In [23]:
# Using the datetime index to calculate means
df_d_wTime = df_h_wTime.resample('D').mean()
df_d_wTime= df_d_wTime.drop(['DTIME','HRMIN'], axis=1)
df_d_wTime['DOY'] = round(df_d_wTime['DOY']).astype('int')

# For Rainf we need to take sum
df_d_wTime['Rainf'] = df_h_wTime[['YEAR', 'DOY', 'Rainf']].resample('D').sum()['Rainf']
df_d_wTime['YEAR'] =  df_d_wTime['YEAR'].astype('int')

# Reset index to columns
df_d = df_d_wTime.reset_index()
df_d = df_d.drop('Time', axis =1)

# Adding the unit row below first row
unit_row = pd.Series([dict_units.get(col, '') for col in df_d.columns], index=df_d.columns)
df_d_save = pd.concat([pd.DataFrame([unit_row]), df_d.iloc[:]]).reset_index(drop=True)

#Saving the dataframes
df_d_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d.csv")
df_d_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d.txt")
fill_value = -6999.
df_d_fv_save = df_d_save.fillna(fill_value)
df_d_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d_fv.csv")
df_d_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d_fv.txt")


## Creating DUKE_forcing_y.txt




In [26]:
# Using the datetime index to calculate means
df_y_wTime = df_d_wTime.resample('Y').mean()
df_y_wTime= df_y_wTime.drop('DOY', axis=1)
df_y_wTime['YEAR'] = round(df_y_wTime['YEAR']).astype('int')

# For Rainf we need to take sum
df_y_wTime['Rainf'] = df_d_wTime[['YEAR', 'Rainf']].resample('Y').sum()['Rainf']
df_y_wTime['YEAR'] =  df_y_wTime['YEAR'].astype('int')

# Reset index to columns
df_y = df_y_wTime.reset_index()
df_y = df_y.drop('Time', axis =1)

# Adding the unit row below first row
unit_row = pd.Series([dict_units.get(col, '') for col in df_y.columns], index=df_y.columns)
df_y_save = pd.concat([pd.DataFrame([unit_row]), df_y.iloc[:]]).reset_index(drop=True)

#Saving the dataframes
df_y_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y.csv")
df_y_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y.txt")
fill_value = -6999.
df_y_fv_save = df_y_save.fillna(fill_value)
df_y_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y_fv.csv")
df_y_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y_fv.txt")

# Making netcdf files


## Creating DUKE_forcing_h.nc

In [102]:
cols_netcdf_h = ['YEAR',
 'DOY',
 'HRMIN',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset

df_h_wTime_fv = df_h_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_h_wTime_fv[cols_netcdf_h])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
ds['DOY'].attrs['units'] = ""
ds['DOY'].attrs['long_name'] = "Day of Measurement"
ds['HRMIN'].attrs['units'] = ""
ds['HRMIN'].attrs['long_name'] = "Hour:minute - marked at the middle of measurement interval with last two digits as minute"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    ds[k_var].attrs['units'] = dict_units[k_var]
    ds[k_var].attrs['missing_value'] = fill_value
    ds[k_var].attrs['long_name'] = dict_cols[k_var]
    ds[k_var].attrs['associate'] = "Time lat lon"
    ds[k_var].attrs['axis'] = "TYX"

# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Half-hourly forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_h.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_h.nc")

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_h.nc


## Creating DUKE_forcing_d.nc

In [103]:
cols_netcdf_d = ['YEAR',
 'DOY',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset
df_h_wTime_fv = df_h_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_h_wTime_fv[cols_netcdf_d])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
ds['DOY'].attrs['units'] = ""
ds['DOY'].attrs['long_name'] = "Day of Measurement"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    ds[k_var].attrs['units'] = dict_units[k_var]
    ds[k_var].attrs['missing_value'] = fill_value
    ds[k_var].attrs['long_name'] = dict_cols[k_var]
    ds[k_var].attrs['associate'] = "Time lat lon"
    ds[k_var].attrs['axis'] = "TYX"

# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Daily forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_h.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_h.nc")

## Creating DUKE_forcing_y.nc

In [107]:
cols_netcdf_y = ['YEAR',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset
df_h_wTime_fv = df_h_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_h_wTime_fv[cols_netcdf_y])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    ds[k_var].attrs['units'] = dict_units[k_var]
    ds[k_var].attrs['missing_value'] = fill_value
    ds[k_var].attrs['long_name'] = dict_cols[k_var]
    ds[k_var].attrs['associate'] = "Time lat lon"
    ds[k_var].attrs['axis'] = "TYX"

# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Yearly forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_h.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_y.nc")

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_y.nc


In [35]:
ds_walker_h = xr.open_dataset(f"{paths['FACEMDS_Walker2018']}DUKE_forcing_h.nc",decode_times=False)
ds_walker_h

<xarray.Dataset>
Dimensions:         (y: 1, x: 1, tstep: 210384)
Dimensions without coordinates: y, x, tstep
Data variables: (12/23)
    nav_lat         (y) float32 ...
    nav_lon         (x) float32 ...
    YEAR            (tstep) int32 ...
    DOY             (tstep) int32 ...
    HRMIN           (tstep) int32 ...
    TIME            (tstep) int32 ...
    ...              ...
    aCO2            (tstep, y, x) float32 ...
    eCO2            (tstep, y, x) float32 ...
    aO3             (tstep, y, x) float32 ...
    eO3             (tstep, y, x) float32 ...
    Ndep            (tstep, y, x) float32 ...
    SolarElevation  (tstep, y, x) float32 ...
Attributes:
    site_id:        DUKE
    title:          Half-hourly forcing data from DUKE Forest FACE, North Car...
    history:        File Origin - This file was created at Oak Ridge National...
    creation_date:  Oct 16, 2014
    contact:        Anthony Walker (walkerap@ornl.gov), Bai Yang (yangb@ornl....

In [96]:
cols_netcdf_h[-10:]

['Rainf', 'Tair', 'RH', 'VPD', 'PAR', 'SM', 'SWP', 'SVP', 'Rn', 'SLT']

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_h.nc


In [85]:
dict_cols['Rainf']

' Precipitation (rainfall + snowfall) rate over a time step of measurement, kg/m2/s'

In [80]:
ds['Time']

<xarray.DataArray 'Time' (Time: 280511)>
array(['1997-01-01T00:30:00.000000000', '1997-01-01T01:00:00.000000000',
       '1997-01-01T01:30:00.000000000', ..., '2012-12-31T22:30:00.000000000',
       '2012-12-31T23:00:00.000000000', '2012-12-31T23:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * Time     (Time) datetime64[ns] 1997-01-01T00:30:00 ... 2012-12-31T23:30:00

In [77]:
ds

<xarray.Dataset>
Dimensions:  (Time: 280511, lon: 1, lat: 1)
Coordinates:
  * Time     (Time) datetime64[ns] 1997-01-01T00:30:00 ... 2012-12-31T23:30:00
  * lat      (lat) float64 35.98
  * lon      (lon) float64 -79.09
Data variables: (12/14)
    YEAR     (lon, lat, Time) int64 1997 1997 1997 1997 ... 2012 2012 2012 2012
    DTIME    (lon, lat, Time) float64 1.021 1.069 1.09 ... 367.5 367.6 367.6
    DOY      (lon, lat, Time) int64 1 1 1 1 1 1 1 ... 366 366 366 366 366 366
    HRMIN    (lon, lat, Time) int64 30 100 130 200 230 ... 2200 2230 2300 2330
    Rainf    (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 0.0 0.0
    Tair     (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 6.88 7.17
    ...       ...
    PAR      (lon, lat, Time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    SM       (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 0.39 0.39
    SWP      (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 488.8 488.8
    SVP      (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 0.99 1.01
    Rn       (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... -31.9 -17.14
    SLT      (lon, lat, Time) float64 -6.999e+03 -6.999e+03 ... 10.4 10.4

In [46]:
# Convert the DataFrame to an xarray Dataset
ds2 = df_w_Time.to_xarray?
ds2

Object `df_w_Time.to_xarray` not found.


NameError: name 'ds2' is not defined

In [ ]:
import pandas as pd


# Create a sample pandas DataFrame
data = {'Time': [1, 2, 3],
        'Value1': [10, 20, 30],
        'Value2': [100, 200, 300]}
df = pd.DataFrame(data)

# Convert the DataFrame to an xarray Dataset
ds = xr.Dataset.from_dataframe(df)

print(ds)
